In [ ]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from itertools import repeat
import random,pickle
def get_centroid(coords_df):
    import math
    x = 0.0
    y = 0.0
    z = 0.0

    for i, coord in coords_df.iterrows():
        latitude = math.radians(coord.latitude)
        longitude = math.radians(coord.longitude)

        x += math.cos(latitude) * math.cos(longitude)
        y += math.cos(latitude) * math.sin(longitude)
        z += math.sin(latitude)

    total = len(coords_df)

    x = x / total
    y = y / total
    z = z / total

    central_longitude = math.atan2(y, x)
    central_square_root = math.sqrt(x * x + y * y)
    central_latitude = math.atan2(z, central_square_root)

    mean_location = {
        'latitude': math.degrees(central_latitude),
        'longitude': math.degrees(central_longitude)
        }
    return mean_location

def zoom_center(lons: tuple=None, lats: tuple=None, lonlats: tuple=None,
        format: str='lonlat', projection: str='mercator',
        width_to_height: float=2.0) -> (float, dict):
   
    if lons is None and lats is None:
        if isinstance(lonlats, tuple):
            lons, lats = zip(*lonlats)
        else:
            raise ValueError(
                'Must pass lons & lats or lonlats'
            )
    
    maxlon, minlon = max(lons), min(lons)
    maxlat, minlat = max(lats), min(lats)
    center = {
        'lon': round((maxlon + minlon) / 2, 6),
        'lat': round((maxlat + minlat) / 2, 6)
    }
    
    # longitudinal range by zoom level (20 to 1)
    # in degrees, if centered at equator
    lon_zoom_range = np.array([
        0.0007, 0.0014, 0.003, 0.006, 0.012, 0.024, 0.048, 0.096,
        0.192, 0.3712, 0.768, 1.536, 3.072, 6.144, 11.8784, 23.7568,
        47.5136, 98.304, 190.0544, 360.0
    ])
    
    if projection == 'mercator':
        margin = 1.2
        height = (maxlat - minlat) * margin * width_to_height
        width = (maxlon - minlon) * margin
        lon_zoom = np.interp(width , lon_zoom_range, range(20, 0, -1))
        lat_zoom = np.interp(height, lon_zoom_range, range(20, 0, -1))
        zoom = round(min(lon_zoom, lat_zoom), 2)
    else:
        raise NotImplementedError(
            f'{projection} projection is not implemented'
        )
    
    return zoom, center
def create_visualization(data_df,user_id,week_num,data_summary_user_current):
 
#     import plotly
    with open('/home/jupyter/sneupane/MOODS/pickled_files/colors.pickle', 'rb') as handle:
        colors = pickle.load(handle)
    random.shuffle(colors)
    with open('../pickled_files/final_df_splitted.pickle', 'rb') as handle:
        final_df_splitted = pickle.load(handle)
    user_df=final_df_splitted.loc[final_df_splitted.user==user_id]
    
    user_df=user_df.loc[(user_df.latitude <=90) & (user_df.latitude>=-90)]
    if len(user_df)>0:
        colors=['#008000', '#457d00', '#647a00', '#7e7500', '#956f00', '#ac6700', '#c15d00', '#d64f00', '#eb3900', '#ff0000']
        mean_location=get_centroid(user_df)
        fig = go.Figure(go.Densitymapbox(lat=user_df.latitude, lon=user_df.longitude, z=user_df.value,colorscale=colors,
                                 zmin=0, zmax=1,colorbar = dict(thickness=7,tick0=0,title="value", dtick=1),
                                ))
#         fig.update_layout(mapbox_style="carto-positron",mapbox_center_lat= mean_location["latitude"],mapbox_center_lon=mean_location["longitude"],mapbox_zoom=12)
        mapboxt =open("/home/jupyter/sneupane/MOODS/visualizations/.mapbox_token").read().strip()
        zoom, mean_location = zoom_center(
        lons=user_df["longitude"],
        lats=user_df["latitude"]
    )
        fig.update_layout(legend=dict(orientation="h", itemclick="toggleothers",
                                  itemdoubleclick="toggle",font=dict(
                    family="Courier",
                    size=10,
                    color="black"
                ),
                                  yanchor="bottom",y=1.05,xanchor="right",x=0.5),autosize=True,showlegend=True,
                                  mapbox = dict(center= dict(lat=mean_location["lat"], lon=mean_location["lon"]),            
                                     accesstoken= mapboxt,
                                     zoom=zoom,
                                     style="carto-positron"
                                   ))
        data_summary_user=data_summary_user_current["Heatmap_Map"] 

       
        fig.update_layout(
                    autosize=True,

                    margin=dict(
                        l=30,
                        r=30,
                        b=10,
                        t=10,
                       
                    ),)
        
        
        return "Heatmap_Map",fig,data_summary_user
    else:
        return "","",""